In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scipy
scipy.__version__

'1.7.0'

In [3]:
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from team36.diff_evo import differential_evolution
from team36.mnist.vgg import VGG
from team36.attacks.fast_gradient_attack_data_set import FastSignGradientAttackDataSet

DIR = '.'
DATA_DIR = f'{DIR}/data'

training_set = torchvision.datasets.MNIST(root=DATA_DIR, train=True, download=True, 
                                          transform=transforms.ToTensor())

prev_model = VGG()
state_dict = torch.load(f"{DIR}/checkpoints/mnist-vgg.pth", map_location=torch.device('cpu'))
print(prev_model.load_state_dict(state_dict))
prev_criterion = nn.CrossEntropyLoss()

attack_training_set = FastSignGradientAttackDataSet(training_set, prev_model, prev_criterion, 
                                                    epsilon=0.25)


indices = torch.randperm(len(attack_training_set))[:int(len(attack_training_set) * 0.1)]
attack_training_set = torch.utils.data.Subset(attack_training_set, indices)

combined_training_set = torch.utils.data.ConcatDataset([training_set, attack_training_set])

training_indices, validation_indices = train_test_split(
    range(len(combined_training_set)),
    test_size=0.1,
)
training_split = torch.utils.data.Subset(combined_training_set, training_indices)
validation_split = torch.utils.data.Subset(combined_training_set, validation_indices)

print(f"{len(training_split)} in training set")
print(f"{len(validation_split)} in validation set")

<All keys matched successfully>
59400 in training set
6600 in validation set


In [113]:
# import argparse
# parser = argparse.ArgumentParser(description='One pixel attack with PyTorch')
# parser.add_argument('--model', default='vgg16', help='The target model')
# parser.add_argument('--pixels', default=1, type=int, help='The number of pixels that can be perturbed.')
# parser.add_argument('--maxiter', default=100, type=int, help='The maximum number of iteration in the DE algorithm.')
# parser.add_argument('--popsize', default=400, type=int, help='The number of adverisal examples in each iteration.')
# parser.add_argument('--samples', default=100, type=int, help='The number of image samples to attack.')
# parser.add_argument('--targeted', action='store_true', help='Set this switch to test for targeted attacks.')
# parser.add_argument('--save', default='./results/results.pkl', help='Save location for the results with pickle.')
# parser.add_argument('--verbose', action='store_true', help='Print out additional information every iteration.')

# args = parser.parse_args()

import os
import sys
import numpy as np

import argparse

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
# from models import *
# from utils import progress_bar
from torch.autograd import Variable
def perturb_image(xs, img):
    if xs.ndim < 2:
        xs = np.array([xs])
    batch = len(xs)
    imgs = img.repeat(batch, 1, 1, 1)
    xs = xs.astype(int)

    count = 0
    for x in xs:
        pixels = np.split(x, len(x)/5)
        
        for pixel in pixels:
            x_pos, y_pos, r, g, b = pixel
            imgs[count, 0, x_pos, y_pos] = (r/255.0-0.4914)/0.2023
            imgs[count, 1, x_pos, y_pos] = (g/255.0-0.4822)/0.1994
            imgs[count, 2, x_pos, y_pos] = (b/255.0-0.4465)/0.2010
        count += 1

    return imgs

def predict_classes(xs, img, target_class, net, minimize=True):
    with torch.no_grad():
        imgs_perturbed = perturb_image(xs, img.clone())
    #     input = Variable(imgs_perturbed, volatile=True).cpu()
        predictions = net(imgs_perturbed).softmax(dim = 1).data.cpu().numpy()[:, target_class]

        return predictions if minimize else 1 - predictions

def attack_success(x, img, target_class, net, targeted_attack=False, verbose=False):
    with torch.no_grad():
        attack_image = perturb_image(x, img.clone())
    #     input = Variable(attack_image, volatile=True).cpu()
        confidence = net(attack_image).softmax(dim = 1).data.cpu().numpy()[0]
        predicted_class = np.argmax(confidence)

        if (verbose):
            print("Confidence: %.4f"%confidence[target_class])
        if (targeted_attack and predicted_class == target_class) or (not targeted_attack and predicted_class != target_class):
            return True


def attack(img, label, net, target=None, pixels=1, maxiter=75, popsize=400, verbose=False):
    # img: 1*3*W*H tensor
    # label: a number
    with torch.no_grad():
        targeted_attack = target is not None
        target_class = target if targeted_attack else label

        bounds = [(0,32), (0,32), (0,255), (0,255), (0,255)] * pixels

        popmul = max(1, popsize/len(bounds))

        predict_fn = lambda xs: predict_classes(
            xs, img, target_class, net, target is None)
        callback_fn = lambda x, convergence: attack_success(
            x, img, target_class, net, targeted_attack, verbose)

        inits = np.zeros([int(popmul)*int(len(bounds)), int(len(bounds))])
        for init in inits:
            for i in range(pixels):
                init[i*5+0] = np.random.random()*32
                init[i*5+1] = np.random.random()*32
                init[i*5+2] = np.random.normal(128,127)
                init[i*5+3] = np.random.normal(128,127)
                init[i*5+4] = np.random.normal(128,127)

        print('starting differential_evolution')
        attack_result = differential_evolution(predict_fn, bounds, maxiter=maxiter, popsize=popmul,
            recombination=1, atol=-1, callback=callback_fn, polish=False, init=inits)
        print("finished differential_evolution")
        attack_image = perturb_image(attack_result.x, img)
        return attack_image
        print(attack_image)
#     #     attack_var = Variable(attack_image, volatile=True).cpu()
#         predicted_probs = net(attack_image).softmax(dim = 1).data.cpu().numpy()[0]

#         predicted_class = np.argmax(predicted_probs)

#         if (not targeted_attack and predicted_class != label) or (targeted_attack and predicted_class == target_class):
#             return 1, attack_result.x.astype(int)
#         return 0, [None]


def attack_all(net, loader, pixels=1, targeted=False, maxiter=75, popsize=400, verbose=False):

    correct = 0
    
    success = 0
    imges = []
#     print(loader.size)
    with torch.no_grad():
        for batch_idx, (input, target) in enumerate(loader):
            print(batch_idx)
#             print(input.shape)
#             print(net(input))
            prior_probs = net(input).softmax(dim = 1)
            _, indices = torch.max(prior_probs, 1)

            if target[0] != indices.data.cpu()[0]:
                continue

            correct += 1
            target = target.numpy()

            targets = [None] if not targeted else range(1)

#             for target_class in targets:
#                 if (targeted):
#                     if (target_class == target[0]):
#                         continue
            target_class = 1
            attack_img = attack(input, target[0], net, target_class, pixels=pixels, maxiter=maxiter, popsize=popsize, verbose=verbose)
            imges.append(attack_img)
#                 success += flag
#                 if (targeted):
#                     success_rate = float(success)/(9*correct)
#                 else:
#                     success_rate = float(success)/correct

#                 if flag == 1:
#                     print("success rate: %.4f (%d/%d) [(x,y) = (%d,%d) and (R,G,B)=(%d,%d,%d)]"%(
#                         success_rate, success, correct, x[0],x[1],x[2],x[3],x[4]))

    #         if correct == args.samples:
    #             break

        return imges


In [114]:

import os
import sys
import numpy as np

import argparse

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
# from models import *
from team36.util import progress_bar
from torch.autograd import Variable

print("==> Loading data and model...")
# tranfrom_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])
# test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tranfrom_test)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True, num_workers=2)

DIR = '.'
DATA_DIR = f'{DIR}/data'
NAME = 'cifar10'

training_set = torchvision.datasets.CIFAR10(root=DATA_DIR, train=True, download=True, 
                                            transform=transforms.ToTensor())

training_indices, validation_indices, _, _ = train_test_split(
    range(len(training_set)),
    training_set.targets,
    stratify=training_set.targets,
    test_size=0.1,
)
training_split = torch.utils.data.Subset(training_set, training_indices)
validation_split = torch.utils.data.Subset(training_set, validation_indices)

print(f"{len(training_split)} in training set")
print(f"{len(validation_split)} in validation set")


import team36
from team36.mnist.vgg import VGG
from team36.training import train, validate

learning_rate = 1e-3
momentum = 5e-1
weight_decay = 5e-2
batch_size = 128
epochs = 1

sampler = torch.utils.data.RandomSampler(training_split, replacement=True, num_samples=1000)
training_loader = torch.utils.data.DataLoader(training_split, batch_size=batch_size, sampler=sampler)
test_loader = torch.utils.data.DataLoader(validation_split, batch_size=100, shuffle=False, num_workers=2)

model = VGG(image_size=32, in_channels=3)
if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                            momentum=momentum, weight_decay=weight_decay)


# class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
# checkpoint = torch.load('./checkpoint/%s.t7'%args.model)
# net = checkpoint['net']
model.cpu()
# cudnn.benchmark = True

print("==> Starting attck...")
pixels = 1
maxiter = 100 
popsize = 400
verbose = True
targeted = True
results = attack_all(model, test_loader, pixels=pixels, targeted=targeted, maxiter=maxiter, popsize=popsize, verbose=verbose)
print("Final success rate: %.4f"%results)

==> Loading data and model...
Files already downloaded and verified
45000 in training set
5000 in validation set
==> Starting attck...
0
1
2
3
4
5
6
starting differential_evolution


KeyboardInterrupt: 